# Build dataset for Disease-symptoms analysis 

In [1]:
import pickle
import re
from googlesearch import search
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup
import time



In [2]:
# Fetch disease list from 'www.nhp.gov.in'
small_alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
diseases=[]
for c in small_alpha:
    URL = 'https://www.nhp.gov.in/disease-a-z/'+c
    time.sleep(1)
    page = requests.get(URL,verify=False)

    soup = BeautifulSoup(page.content, 'html5lib')
    all_diseases = soup.find('div', class_='all-disease')

    for dis in all_diseases.find_all('li'):
        diseases.append(dis.get_text().strip())



In [3]:
len(diseases)

303

In [7]:
pickle.dump(diseases,open("diseaseNames.pkl",'wb'))

In [8]:
diseases2=pickle.load(open("diseaseNames.pkl",'rb'))

# combine diseases and symptoms

In [38]:
from tqdm import tqdm 

In [21]:

dis_symp={}

for dis in tqdm(diseases2):
    query = dis+' wikipedia'
 
    for sr in search(query,tld="co.in",stop=10,pause=0.5): 
       
        match=re.search(r'wikipedia',sr)
        filled = 0
        if match:
            wiki = requests.get(sr,verify=False)
            soup = BeautifulSoup(wiki.content, 'html5lib')
   
            info_table = soup.find("table", {"class":"infobox"})
            if info_table is not None:
             
                for row in info_table.find_all("tr"):
                    data=row.find("th",{"scope":"row"})
                    if data is not None:
                        data=data.get_text()
                        if data=="Symptoms":
                            symptom=str(row.find("td"))
                            symptom = symptom.replace('.','')
                            symptom = symptom.replace(';',',')
                            symptom=re.sub(r'<b.*?/b>:',',',symptom) # Remove bold text
                            symptom=re.sub(r'<a.*?>','',symptom) # Remove hyperlink
                            symptom=re.sub(r'</a>','',symptom) # Remove hyperlink
                            symptom=re.sub(r'<[^<]+?>',', ',symptom) # All the tags
                            symptom=re.sub(r'\[.*\]','',symptom) # Remove citation text
                            symptom=' '.join([x for x in symptom.split() if x != ','])
                            dis_symp[dis]=symptom
                     
                            filled = 1
                            break
        if filled==1:
            break
      


100%|██████████| 303/303 [32:15<00:00,  6.39s/it]  


In [29]:
len(dis_symp)

202

In [28]:
#pickle.dump(dis_symp,open("Disease-symptoms.pkl",'wb'))

In [35]:
#remove diseases that have same symptoms

temp_list=[]
tmp_dict=dict()
for key,value in dis_symp.items():
  if value not in temp_list:
    tmp_dict[key]=value
    temp_list.append(value)
  else:
    print(key)

Anaemia during pregnancy (Maternal anemia)
Crimean Congo haemorrhagic fever (CCHF)
Congenital Capillary Haemangioma of Eyelid
Cavernous Haemangioma of Eyelid
Epidemic Keratoconjunctivitis
Giant Papillary Conjunctivitis
Hypertensive Retinopathy
Hypertension (High Blood pressure)
Human papillomavirus (HPV) infection and cervical cancer
Low Vision and VisuaI Aids
Long COVID/Post-COVID conditions
Neonatal Conjunctivitis
Orbital Haemangiopericytoma
Post Menopausal Bleeding
Post Polio Syndrome
Perennial Allergic Conjunctivitis
Pinguecula
Seasonal Allergic Conjunctivitis
Toxic Keratoconjunctivitis
Thermal Injuries to the Eye
Vernal Keratoconjunctivitis
Whooping Cough/Pertussis


In [33]:
len(tmp_dict)

180

In [36]:
pickle.dump(tmp_dict,open("Disease-symptoms.pkl",'wb'))